In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.configuration.acidity import ForcingParameter
from seapopym.configuration.no_transport import ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.cost_function import SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import AcidityFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.genetic_algorithm.simple_genetic_algorithm import (
    SimpleGeneticAlgorithm,
    SimpleGeneticAlgorithmParameters,
)
from seapopym_optimization.model_generator import AcidityModelGenerator


In [3]:
import logging

logging.basicConfig(level=logging.WARNING, format="\n%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("seapopym_optimization")
logger.setLevel(logging.DEBUG)

In [4]:
fg1 = AcidityFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=1),
    lambda_acidity_0=Parameter(name="lambda_acidity_0", lower_bound=0, upper_bound=1),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    gamma_lambda_acidity=Parameter(name="gamma_lambda_acidity", lower_bound=0, upper_bound=1),
)

fg2 = AcidityFunctionalGroup(
    name="FG2",
    day_layer=1,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=1),
    lambda_acidity_0=Parameter(name="lambda_acidity_0", lower_bound=0, upper_bound=1),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    gamma_lambda_acidity=Parameter(name="gamma_lambda_acidity", lower_bound=0, upper_bound=1),
)

In [5]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_transfert", "FG2_energy_transfert"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [6]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)

In [7]:
fg_set.unique_functional_groups_parameters_ordered().keys()

dict_keys(['FG1_energy_transfert', 'lambda_temperature_0', 'gamma_lambda_temperature', 'tr_0', 'gamma_tr', 'lambda_acidity_0', 'gamma_lambda_acidity', 'FG2_energy_transfert'])

In [9]:
parameters_set = [1, np.inf, 1, 1, -1, 1, 1, 1]
functional_group_parameters = fg_set.generate(parameters_set)
functional_group_parameters

[{'day_layer': 0,
  'night_layer': 0,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1,
  'lambda_acidity_0': 1,
  'gamma_lambda_acidity': 1},
 {'day_layer': 1,
  'night_layer': 1,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1,
  'lambda_acidity_0': 1,
  'gamma_lambda_acidity': 1}]

In [12]:
nb_days = 360
temperature = xr.DataArray(
    data=np.zeros((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
acidity = xr.DataArray(
    data=np.zeros((nb_days, 1, 1, 2)) * 8,
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="acidity",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=np.ones((nb_days, 1, 1)),
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "kg/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
observation = xr.DataArray(
    data=np.ones((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "layer"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "layer": [0, 1],
    },
    attrs={
        "units": "kg/m^2",
    },
)
observation.time.attrs = {"axis": "T"}
temperature.time.attrs = {"axis": "T"}
acidity.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}

observation.latitude.attrs = {"axis": "Y"}
temperature.latitude.attrs = {"axis": "Y"}
acidity.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}

observation.longitude.attrs = {"axis": "X"}
temperature.longitude.attrs = {"axis": "X"}
acidity.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

observation.layer.attrs = {"axis": "Z"}
temperature.depth.attrs = {"axis": "Z"}
acidity.depth.attrs = {"axis": "Z"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
    acidity=ForcingUnit(forcing=acidity),
)
with xr.set_options(keep_attrs=True):
    observation_2 = observation.sel(layer=[1]) / 2

In [13]:
observation_1 = TimeSeriesObservation(
    name="Observation_all",
    observation=observation,
)
observation_2 = TimeSeriesObservation(
    name="Observation_meso",
    observation=observation_2,
)

In [14]:
model_generator = AcidityModelGenerator(forcing_parameters=forcing_parameter)

In [15]:
model = model_generator.generate(functional_group_parameters=functional_group_parameters)

In [16]:
model.state

<xarray.Dataset> Size: 18kB
Dimensions:                     (time: 360, latitude: 1, longitude: 1,
                                 depth: 2, functional_group: 2, cohort: 1)
Coordinates:
  * latitude                    (latitude) int64 8B 0
  * longitude                   (longitude) int64 8B 0
  * depth                       (depth) int64 16B 0 1
  * time                        (time) datetime64[ns] 3kB 2023-01-01 ... 2023...
  * cohort                      (cohort) int64 8B 0
  * functional_group            (functional_group) int64 16B 0 1
Data variables: (12/21)
    temperature                 (time, latitude, longitude, depth) float64 6kB ...
    primary_production          (time, latitude, longitude) float64 3kB 1.0 ....
    acidity                     (time, latitude, longitude, depth) float64 6kB ...
    name                        (functional_group) <U7 56B 'Group_0' 'Group_1'
    energy_transfert            (functional_group) int64 16B 1 1
    lambda_temperature_0        (functional_group) float64 16B inf inf
    ...                          ...
    max_timestep                (functional_group, cohort) int64 16B 1 1
    mean_timestep               (functional_group, cohort) float64 16B 1.0 1.0
    timestep                    int64 8B 1
    angle_horizon_sun           float64 8B 0.0
    compute_initial_conditions  bool 1B False
    compute_preproduction       bool 1B False

In [ ]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[observation_1, observation_2],
    functional_groups=[fg1, fg2],
    root_mse=True,
    centered_mse=False,
    normalized_mse=False,
)
partial_cost_function = cost_function.generate()
partial_cost_function(parameters_set)

(np.float64(0.0), np.float64(0.5))

In [18]:
metaparam = SimpleGeneticAlgorithmParameters(
    ETA=4, INDPB=0.2, CXPB=0.5, MUTPB=0.2, NGEN=3, POP_SIZE=10, cost_function_weight=(-1, -1)
)
genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam, cost_function=cost_function, constraint=[constraint]
)

In [19]:
viewer = genetic_algorithm.optimize()

In [20]:
genetic_algorithm.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.786146   
                                       1                      0.000284   
                                       2                      0.582955   
                                       3                      0.626718   
                                       4                      0.119551   
                                       5                      0.574111   
                                       6                      0.034144   
                                       7                      0.635785   
                                       8                      0.245183   
                                       9                      0.801596   
1          False                       0                      0.000284   
                                       1                      0.119551   
           True                        2                      0.574111   
                                       3                      0.034144   
           False                       4                      0.582955   
                                       5                      0.574111   
           True                        6                      0.582955   
                                       7                      0.582955   
                                       8                      0.034144   
                                       9                      0.582955   
2          False                       0                      0.582955   
                                       1                      0.574111   
           True                        2                      0.574111   
                                       3                      0.582955   
                                       4                      0.582955   
                                       5                      0.574111   
                                       6                      0.574111   
                                       7                      0.582955   
                                       8                      0.582955   
           False                       9                      0.034144   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.876023   
                                       1                      0.399106   
                                       2                      0.071437   
                                       3                      0.045400   
                                       4                      0.489295   
                                       5                      0.441515   
                                       6                      0.757793   
                                       7                      0.070075   
                                       8                      0.322185   
                                       9                      0.979059   
1          False                       0                      0.399106   
                                       1                      0.489295   
           True                        2                      0.441515   
                                       3                      0.757793   
           False                       4                      0.441515   
                                       5                      0.071437   
           True                        6                      0.071437   
                                       7                      0.071437   
   

In [21]:
unevaluated_logbook = genetic_algorithm.logbook.loc[[0]].copy()
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.786146   
                                       1                      0.000284   
                                       2                      0.582955   
                                       3                      0.626718   
                                       4                      0.119551   
                                       5                      0.574111   
                                       6                      0.034144   
                                       7                      0.635785   
                                       8                      0.245183   
                                       9                      0.801596   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.876023   
                                       1                      0.399106   
                                       2                      0.071437   
                                       3                      0.045400   
                                       4                      0.489295   
                                       5                      0.441515   
                                       6                      0.757793   
                                       7                      0.070075   
                                       8                      0.322185   
                                       9                      0.979059   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.805319   
                                       1                          0.517218   
                                       2                          0.299954   
                                       3                          0.975150   
                                       4                          0.037271   
                                       5                          0.401801   
                                       6                          0.607417   
                                       7                          0.126621   
                                       8                          0.330291   
                                       9                          0.264739   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           7.143927 -0.238342   
                                       1           4.021467 -0.249904   
                                       2           9.461591 -0.448306   
                                       3           3.966263 -0.771807   
                                       4           9.552544 -0.814631   
                                       5           9.230558 -0.348796   
                                       6           3.884936 -0.786184   
                                       7           6.752374 -0.465892   
                                       8           1.628416 -0.675659   
                                       9           9.221153 -0.768898   

category                                                            \
name                                   

In [22]:
from seapopym_optimization.genetic_algorithm.simple_logbook import Logbook


unevaluated_logbook.iloc[:5, -3:] = np.nan
unevaluated_logbook = Logbook(unevaluated_logbook)
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.786146   
                                       1                      0.000284   
                                       2                      0.582955   
                                       3                      0.626718   
                                       4                      0.119551   
                                       5                      0.574111   
                                       6                      0.034144   
                                       7                      0.635785   
                                       8                      0.245183   
                                       9                      0.801596   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.876023   
                                       1                      0.399106   
                                       2                      0.071437   
                                       3                      0.045400   
                                       4                      0.489295   
                                       5                      0.441515   
                                       6                      0.757793   
                                       7                      0.070075   
                                       8                      0.322185   
                                       9                      0.979059   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.805319   
                                       1                          0.517218   
                                       2                          0.299954   
                                       3                          0.975150   
                                       4                          0.037271   
                                       5                          0.401801   
                                       6                          0.607417   
                                       7                          0.126621   
                                       8                          0.330291   
                                       9                          0.264739   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           7.143927 -0.238342   
                                       1           4.021467 -0.249904   
                                       2           9.461591 -0.448306   
                                       3           3.966263 -0.771807   
                                       4           9.552544 -0.814631   
                                       5           9.230558 -0.348796   
                                       6           3.884936 -0.786184   
                                       7           6.752374 -0.465892   
                                       8           1.628416 -0.675659   
                                       9           9.221153 -0.768898   

category                                                            \
name                                   

In [23]:
new_genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam,
    cost_function=cost_function,
    constraint=[constraint],
    logbook=unevaluated_logbook,
)


In [24]:
viewer = new_genetic_algorithm.optimize()


2025-06-10 14:01:32,844 - seapopym_optimization.genetic_algorithm.simple_genetic_algorithm - INFO - Logbook found. Loading last generation.

2025-06-10 14:01:32,846 - seapopym_optimization.genetic_algorithm.simple_genetic_algorithm - WARNING - Some individuals in the logbook have no fitness values. Re-evaluating the population.


In [25]:
viewer.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.786146   
                                       1                      0.000284   
                                       2                      0.582955   
                                       3                      0.626718   
                                       4                      0.119551   
           True                        5                      0.574111   
                                       6                      0.034144   
                                       7                      0.635785   
                                       8                      0.245183   
                                       9                      0.801596   
1          False                       0                      0.635785   
                                       1                      0.679555   
           True                        2                      0.574111   
                                       3                      0.574111   
                                       4                      0.034144   
           False                       5                      0.574111   
           True                        6                      0.582955   
                                       7                      0.034144   
                                       8                      0.582955   
                                       9                      0.574111   
2          False                       0                      0.574111   
                                       1                      0.574111   
                                       2                      0.574111   
                                       3                      0.574111   
                                       4                      0.574111   
                                       5                      0.574111   
           True                        6                      0.574111   
                                       7                      0.574111   
                                       8                      0.574111   
                                       9                      0.574111   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.876023   
                                       1                      0.399106   
                                       2                      0.071437   
                                       3                      0.045400   
                                       4                      0.489295   
           True                        5                      0.441515   
                                       6                      0.757793   
                                       7                      0.070075   
                                       8                      0.322185   
                                       9                      0.979059   
1          False                       0                      0.070075   
                                       1                      0.071437   
           True                        2                      0.441515   
                                       3                      0.441515   
                                       4                      0.757793   
           False                       5                      0.441515   
           True                        6                      0.071437   
                                       7                      0.757793   
   

In [26]:
viewer.stats()

,mean,std,min,max,valid,from_previous_generation
Generation,,,,,,
0,-0.484028,0.276783,-1.030322,-0.253498,6,0.5
1,-0.345444,0.086161,-0.462162,-0.253498,10,0.7
2,-0.253846,0.001098,-0.256972,-0.253498,10,0.4


In [27]:
viewer.hall_of_fame()

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          True                        5                      0.574111   
1          False                       5                      0.574111   
2          False                       4                      0.574111   
0          True                        7                      0.635785   
           False                       2                      0.582955   
1          False                       1                      0.679555   
                                       0                      0.635785   
0          False                       4                      0.119551   
           True                        6                      0.034144   
           False                       1                      0.000284   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          True                        5                      0.441515   
1          False                       5                      0.441515   
2          False                       4                      0.316656   
0          True                        7                      0.070075   
           False                       2                      0.071437   
1          False                       1                      0.071437   
                                       0                      0.070075   
0          False                       4                      0.489295   
           True                        6                      0.757793   
           False                       1                      0.399106   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          True                        5                          0.401801   
1          False                       5                          0.401801   
2          False                       4                          0.401801   
0          True                        7                          0.126621   
           False                       2                          0.299954   
1          False                       1                          0.199499   
                                       0                          0.126621   
0          False                       4                          0.037271   
           True                        6                          0.607417   
           False                       1                          0.517218   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          True                        5           9.230558 -0.348796   
1          False                       5           9.230558 -0.348796   
2          False                       4           9.230558 -0.348796   
0          True                        7           6.752374 -0.465892   
           False                       2           9.461591 -0.448306   
1          False                       1           9.798196 -0.525142   
                                       0           6.752374 -0.448306   
0          False                       4           9.552544 -0.814631   
           True                        6           3.884936 -0.786184   
           False                       1           4.021467 -0.249904   

category                                                            \
name                                   

In [28]:
viewer.fitness_evolution(points="all")

In [29]:
viewer.box_plot(3)

In [30]:
viewer.parallel_coordinates()[0]

In [31]:
viewer.parameters_standardized_deviation()

In [32]:
viewer.shannon_entropy(bins=100_000)

In [33]:
for fig in viewer.time_series(10):
    fig.show()